#### 지역의 날씨 정보 조회
##### 1. 지역의 위경도 조회 API 
##### 2. 지역의 날씨 조회 API 

In [ ]:
import os
import json
import requests
from openai import OpenAI
from datetime import datetime
# zoneinfo는 Windows에서 IANA Timezone 데이터를 사용하기 위해 필요합니다.
from zoneinfo import ZoneInfo 

# 환경 변수 로드를 위한 dotenv 모듈
from dotenv import load_dotenv

# .env 환경 변수 로드
load_dotenv()

# --- 1. 환경 변수 설정 및 클라이언트 초기화 ---
# Azure OpenAI 설정 (엔드포인트는 클라이언트 초기화에서 바로 사용)
AZURE_OAI_KEY = os.getenv("AZURE_OAI_KEY")
AZURE_OAI_DEPLOYMENT = os.getenv("AZURE_OAI_DEPLOYMENT")
# OpenWeatherMap API 키 (날씨 정보 조회에 사용)
OPEN_WEATHER_API_KEY = os.getenv("OPEN_WEATHER_API_KEY")

DEPLOYMENT_NAME = AZURE_OAI_DEPLOYMENT
BASE_URL = "https://8ai022-openai.openai.azure.com/openai/v1/" # 하드코딩된 URL 사용

# Azure OpenAI 클라이언트 초기화
try:
    client = OpenAI(
        base_url=BASE_URL, # API 엔드포인트
        api_key=AZURE_OAI_KEY # API 키
    )
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    # 프로그램 실행을 멈추기 위해 종료
    exit()

# --- 2. 보조 데이터 (타임존 정보) ---
# 시간 조회를 위한 주요 도시와 타임존 매핑
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris",
    "seoul": "Asia/Seoul"
}


# --- 3. Function Calling에 사용될 함수 정의 (도구) ---

def get_current_weather(location: str, unit: str = "celsius") -> str:
    """
    주어진 도시의 현재 날씨 정보를 조회합니다.
    location은 도시 이름이며, unit은 온도 단위입니다 (기본값: celsius).
    """
    print(f"\n--- [Tool Call: get_current_weather] ---")
    print(f"Location: {location}, Unit: {unit}")
    
    # API 키가 환경 변수에 설정되어 있는지 확인
    if not OPEN_WEATHER_API_KEY:
        error_message = "OpenWeatherMap API Key가 설정되지 않았습니다."
        print(error_message)
        return json.dumps({"error": error_message})

    # 1단계: 도시 이름으로 위도 및 경도(Latitude, Longitude) 조회 (Geocoding API)
    # OpenWeatherMap API 키 변수 사용
    geo_endpoint = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&appid={OPEN_WEATHER_API_KEY}"
    try:
        geo_response = requests.get(geo_endpoint, timeout=5)
        geo_response.raise_for_status() # HTTP 오류 발생 시 예외 발생
        
        geo_json = geo_response.json()
        if not geo_json:
            return json.dumps({"location": location, "weather": "City not found."})
        
        # 첫 번째 검색 결과 사용
        lat = geo_json[0].get('lat')
        lon = geo_json[0].get('lon')
        
        if lat is None or lon is None:
            return json.dumps({"location": location, "weather": "Invalid coordinates received."})

        # 2단계: 위경도를 사용하여 현재 날씨 정보 조회
        units_param = "metric" if unit.lower() == "celsius" else "imperial"
        # OpenWeatherMap API 키 변수 사용
        weather_endpoint = (
            f"https://api.openweathermap.org/data/2.5/weather?"
            f"units={units_param}&lat={lat}&lon={lon}&appid={OPEN_WEATHER_API_KEY}"
        )

        weather_response = requests.get(weather_endpoint, timeout=5)
        weather_response.raise_for_status()
        
        weather_data = weather_response.json()
        
        # 필요한 정보 추출
        temperature = weather_data.get('main', {}).get('temp')
        weather_desc = weather_data.get('weather', [{}])[0].get('description')
        
        result = {
            "location": weather_data.get('name', location),
            "temperature": temperature,
            "unit": unit.lower(),
            "description": weather_desc
        }
        return json.dumps(result)

    except requests.exceptions.Timeout:
        return json.dumps({"location": location, "error": "API request timed out."})
    except requests.exceptions.RequestException as e:
        print(f"Weather API request failed: {e}")
        return json.dumps({"location": location, "error": f"API request failed: {e}"})
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return json.dumps({"location": location, "error": f"An unexpected error occurred: {e}"})


def get_current_time(location: str) -> str:
    """
    주어진 도시의 현재 시간을 조회합니다.
    """
    print(f"\n--- [Tool Call: get_current_time] ---")
    print(f"Location: {location}")

    location_lower = location.lower()
    
    # 모델이 추출한 도시 이름을 기반으로 타임존 매핑 데이터 검색
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            try:
                # zoneinfo를 사용하여 현재 시간 객체 생성 및 포맷팅
                current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
                return json.dumps({
                    "location": location,
                    "current_time": current_time,
                    "timezone": timezone
                })
            except Exception as e:
                # tzdata 패키지 미설치 등으로 ZoneInfo 오류 발생 시
                return json.dumps({"location": location, "error": f"Timezone error: {e}. (Need 'pip install tzdata' on Windows)"})

    # 매핑 데이터에 없는 도시인 경우
    print(f"No timezone data found for {location_lower}") 
    return json.dumps({"location": location, "current_time": "unknown"})


# --- 4. 메인 대화 실행 함수 ---

def run_conversation() -> str:
    # 챗 메시지 기록 (history) 초기화
    # 한글 사용자 입력 설정
    messages = [{"role": "user", "content": "서울의 날씨 정보와 현재 시간을 알려줘"}] 

    # 모델에 전달할 함수 정의 목록 (tools)
    tools = [
        # 1. 날씨 조회 함수 정의
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "주어진 도시의 현재 날씨 정보를 조회합니다. 한글 도시명도 처리 가능합니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "조회할 도시 이름 (예: 서울, Tokyo, 파리)",
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "온도 단위. 기본값은 celsius(섭씨)입니다."
                        }
                    },
                    "required": ["location"],
                },
            }
        },
        # 2. 시간 조회 함수 정의
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "주어진 도시의 현재 시간을 조회합니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "조회할 도시 이름 (예: 서울, San Francisco)",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

    # 1차 API 호출: 모델이 함수 호출을 결정하고 JSON 응답을 반환
    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
            tools=tools,
            tool_choice="auto", # 모델이 함수 호출 여부를 자동으로 결정
        )
    except Exception as e:
        return f"Azure OpenAI API 호출 중 오류 발생: {e}"

    response_message = response.choices[0].message
    messages.append(response_message) # 모델의 함수 호출 요청 메시지를 기록에 추가

    print("\n--- [LLM 1차 응답 (함수 호출 요청)] ---") 
    print(response_message) 

    # 함수 호출 처리
    if response_message.tool_calls:
        tool_function_map = {
            "get_current_weather": get_current_weather,
            "get_current_time": get_current_time
        }
        
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = tool_function_map.get(function_name)
            
            if function_to_call:
                # LLM이 생성한 JSON 인수를 파이썬 딕셔너리로 변환
                function_args = json.loads(tool_call.function.arguments)
                
                # 실제 함수 실행 및 결과 얻기
                function_response = function_to_call(**function_args)
                
                # 함수 실행 결과를 메시지 기록에 추가 (role: 'tool')
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                })
            else:
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps({"error": f"Unknown function: {function_name}"}),
                })
    else:
        print("\nNo tool calls were made by the model. (직접 텍스트 응답)") 
        return response_message.content

    # 2차 API 호출: 함수 실행 결과(content)를 바탕으로 최종 답변 생성
    final_response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages, # 1차 응답 + 함수 결과가 모두 포함된 메시지
    )

    print("\n--- [LLM 2차 응답 (최종 자연어 답변)] ---") 
    return final_response.choices[0].message.content

# --- 5. 프로그램 실행 ---

# 사용자가 한글로 입력하는 요청
korean_prompt = "서울의 날씨 정보와 현재 시간을 알려줘" 

print(f"사용자 입력: {korean_prompt}")
result = run_conversation() # run_conversation 함수는 내부적으로 korean_prompt를 사용하도록 수정됨

# 최종 결과 출력
print("-" * 50)
print(f"최종 답변: {result}")
print("-" * 50)

사용자 입력: sds

--- [LLM 1차 응답 (함수 호출 요청)] ---
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_ir0QPq6xoWm2p9Yp7Vxnb3Bz', function=Function(arguments='{"location": "서울"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_LwXBdL4N9Zj3BbIVArtttKvD', function=Function(arguments='{"location": "서울"}', name='get_current_time'), type='function')])

--- [Tool Call: get_current_weather] ---
Location: 서울, Unit: celsius

--- [Tool Call: get_current_time] ---
Location: 서울
No timezone data found for 서울

--- [LLM 2차 응답 (최종 자연어 답변)] ---
--------------------------------------------------
최종 답변: 현재 서울의 날씨는 다음과 같습니다:
- **온도**: 8.82°C
- **날씨**: 강한 비가 내리고 있습니다.

서울의 현재 시간을 확인할 수 없습니다. 다른 요청이 있으시면 말씀해 주세요!
--------------------------------------------------
